# Introduction to Task Planning

In this example, we will describe a simple robot pick and place domain.
Suppose we have a robot that is picking up parts and putting them into bins.

Before you start, make sure you have installed the Unified Planning library:

```
pip install unified_planning[tamer]
```

Here, the `[tamer]` addon means that, in addition to installing the core Unified Planning library, we're also installing the Tamer planner so we can actually do planning.

In [28]:
from unified_planning.shortcuts import *

We will first create a set of **types** for our domain.
In this case, we care about two types of objects: parts and bins. 

In [2]:
Part = UserType("part")
Bin = UserType("bin")

Roughly, the rules are as follows:
* The robot needs to move to a bin before picking/placing a part.
* The robot can only be holding one part at a time.
* Each bin can only hold one part.

To capture these requirements, we can create a set of **fluents** that describe the world at this level of abstraction.

In [3]:
robot_at = Fluent("robot_at", BoolType(), b=Bin)
part_at = Fluent("part_at", BoolType(), p=Part, b=Bin)
holding = Fluent("holding", BoolType(), p=Part)
gripper_empty = Fluent("gripper_empty", BoolType())

These fluents are used to define the possible **actions** in our domain.

In this case, we will have 3 actions: *move*, *pick*, and *place*.

Each action contains a set of **preconditions** (requirements to run the action) and **effects** (what happens to the fluents when the action is performed).

In [4]:
move = InstantaneousAction("move", b_from=Bin, b_to=Bin)
b_from = move.parameter("b_from")
b_to = move.parameter("b_to")
move.add_precondition(robot_at(b_from))
move.add_effect(robot_at(b_from), False)
move.add_effect(robot_at(b_to), True)
#print(move)

pick = InstantaneousAction("pick", p=Part, b=Bin)
p = pick.parameter("p")
b = pick.parameter("b")
pick.add_precondition(robot_at(b))
pick.add_precondition(part_at(p, b))
pick.add_precondition(gripper_empty)
pick.add_effect(part_at(p, b), False)
pick.add_effect(holding(p), True)
pick.add_effect(gripper_empty, False)
#print(pick)

place = InstantaneousAction("place", p=Part, b=Bin)
p = place.parameter("p")
b = place.parameter("b")
place.add_precondition(robot_at(b))
place.add_precondition(holding(p))
place.add_precondition(Not(gripper_empty))
place.add_effect(part_at(p, b), True)
place.add_effect(holding(p), False)
place.add_effect(gripper_empty, True)
#print(place)

Now that we have types, fluents, and actions, our entire **planning domain** is specified.

However, we can't do anything with this abstract description until we define a set of **objects** -- that is, real instances of our specified types that can be used for planning.

Let's define a few parts and bins for our problem.

In [5]:
part_1 = Object("part_1", Part)
part_2 = Object("part_2", Part)
bin_1 = Object("bin_1", Bin)
bin_2 = Object("bin_2", Bin)
bin_3 = Object("bin_3", Bin)

Let's now assemble a full **problem**, which comprises

* The domain (types, fluents, and actions)
* The objects
* The initial conditions
* The desired goal state

In [25]:
problem = Problem("pick_and_place")
problem.add_fluent(robot_at, default_initial_value=False)
problem.add_fluent(part_at, default_initial_value=False)
problem.add_fluent(holding, default_initial_value=False)
problem.add_fluent(gripper_empty, default_initial_value=False)
problem.add_actions([move, pick, place])
problem.add_objects([part_1, part_2, bin_1, bin_2, bin_3])

# Initial conditions
problem.set_initial_value(gripper_empty, True)
problem.set_initial_value(robot_at(bin_1), False)
problem.set_initial_value(robot_at(bin_3), True)
problem.set_initial_value(part_at(part_1, bin_1), True)
problem.set_initial_value(part_at(part_2, bin_2), True)

# Desired goal state
problem.add_goal(robot_at(bin_2))
problem.add_goal(part_at(part_1, bin_3))
problem.add_goal(part_at(part_2, bin_1))

#print(problem)

Now that we have this problem set up, we can **plan** to solve it!

In [27]:
with OneshotPlanner(name="tamer") as planner:
    result = planner.solve(problem)
    if result.plan is None:
        print("Found no plan :(")
    else:
        print(result.plan)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_2327289/1018834468.py`, you are using the following planning engine:
  * Engine name: Tamer
  * Developers:  FBK Tamer Development Team
  * Description: Tamer offers the capability to generate a plan for classical, numerical and temporal problems.
  *              For those kind of problems tamer also offers the possibility of validating a submitted plan.

SequentialPlan:
    move(bin_3, bin_2)
    pick(part_2, bin_2)
    move(bin_2, bin_1)
    place(part_2, bin_1)
    pick(part_1, bin_1)
    move(bin_1, bin_3)
    place(part_1, bin_3)
    move(bin_3, bin_2)


... and there we go -- our first task planning example!

Feel free to change the number of objects, initial conditions, and/or goals to see how the planner performs.

This is why it can be useful to go through the trouble of setting up automated planning problems like this one; because once you define the structure of the problem, you can find plans for any arbitrary combination of objects that fit within this domain specification.